<a href="https://colab.research.google.com/github/Vishwaak/Deep_Fun/blob/master/One_Shot_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from imageio import imread

In [0]:
def initialize_weights(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [0]:
def initialize_bias(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)


In [4]:
input_shape = (105,105,1)
left_input = Input(input_shape)
right_input = Input(input_shape)

In [0]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [6]:
model = get_siamese_model((105, 105, 1))
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         38947648    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           sequential_1[1][0]       

In [8]:
!git clone https://github.com/brendenlake/omniglot.git
!unzip /content/omniglot/python/images_background.zip
!unzip /content/omniglot/python/images_evaluation.zip
!mkdir data

Streaming output truncated to the last 5000 lines.
  inflating: images_evaluation/Mongolian/character28/1386_06.png  
  inflating: images_evaluation/Mongolian/character28/1386_07.png  
  inflating: images_evaluation/Mongolian/character28/1386_08.png  
  inflating: images_evaluation/Mongolian/character28/1386_09.png  
  inflating: images_evaluation/Mongolian/character28/1386_10.png  
  inflating: images_evaluation/Mongolian/character28/1386_11.png  
  inflating: images_evaluation/Mongolian/character28/1386_12.png  
  inflating: images_evaluation/Mongolian/character28/1386_13.png  
  inflating: images_evaluation/Mongolian/character28/1386_14.png  
  inflating: images_evaluation/Mongolian/character28/1386_15.png  
  inflating: images_evaluation/Mongolian/character28/1386_16.png  
  inflating: images_evaluation/Mongolian/character28/1386_17.png  
  inflating: images_evaluation/Mongolian/character28/1386_18.png  
  inflating: images_evaluation/Mongolian/character28/1386_19.png  
  inflating

In [0]:
train_folder = '/content/images_background'
val_folder =  '/content/images_evaluation'

In [0]:
def image_load(path,n = 0):
    X=[]
    y=[]
    cat_dict = {}
    lang_dict = {}
    curr_y=n
    for alphabet in os.listdir(path):
        print("Loading " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        for letter in os.listdir(alphabet_path):
            cat_dict[curr_y] = (alphabet , letter)
            category_image=[]
            letter_path = os.path.join(alphabet_path,letter)
            for filename in os.listdir(letter_path):
                image_path = os.path.join(letter_path,filename)
                image = imread(image_path)
                category_image.append(image)
                y.append(curr_y)
                X.append(np.stack(category_image))
            curr_y+=1
            lang_dict[alphabet][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(y)
    return X,y,lang_dict

In [25]:
X,y,C = image_load(train_folder)

Loading Latin
Loading Grantha
Loading Greek
Loading Burmese_(Myanmar)
Loading Syriac_(Estrangelo)
Loading Hebrew
Loading Bengali
Loading Alphabet_of_the_Magi
Loading Blackfoot_(Canadian_Aboriginal_Syllabics)
Loading Balinese
Loading Gujarati
Loading Sanskrit
Loading Malay_(Jawi_-_Arabic)
Loading Arcadian
Loading Inuktitut_(Canadian_Aboriginal_Syllabics)
Loading Braille
Loading Japanese_(hiragana)
Loading Ojibwe_(Canadian_Aboriginal_Syllabics)
Loading Armenian
Loading Early_Aramaic
Loading Japanese_(katakana)
Loading Tifinagh
Loading Futurama
Loading Mkhedruli_(Georgian)
Loading Tagalog
Loading Anglo-Saxon_Futhorc
Loading Cyrillic
Loading Asomtavruli_(Georgian)
Loading Korean
Loading N_Ko


In [38]:
X_val , y_val, c_val = image_load(val_folder)

Loading Sylheti
Loading Atemayar_Qelisayer
Loading Aurek-Besh
Loading Syriac_(Serto)
Loading Tibetan
Loading Keble
Loading Avesta
Loading Gurmukhi
Loading Manipuri
Loading Tengwar
Loading ULOG
Loading Mongolian
Loading Kannada
Loading Atlantean
Loading Ge_ez
Loading Malayalam
Loading Old_Church_Slavonic_(Cyrillic)
Loading Oriya
Loading Angelic
Loading Glagolitic


In [0]:
import seaborn as sns
import pandas as pd
df = pd.DataFrame(c_val.values(),c_val.keys())
